In [9]:
import csv
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import joblib

file_feature = "./csv/endometrium.csv"
file_train = "./csv/train.csv"
file_validate = "./csv/validation.csv"
file_test = "./csv/test.csv"

f = open(file_feature)
csv_f = csv.reader(f)
features = next(csv_f)
dataset = pd.read_csv(file_feature, names=features, usecols=range(1,6098), dtype=np.float64, skiprows=1, low_memory=False)
f = open(file_train)
csv_f = csv.reader(f)
features = next(csv_f)
dataset_train = pd.read_csv(file_train, names=features, usecols=range(1,1), dtype=np.float64, skiprows=1, low_memory=False)

with open('./csv/train.csv','r') as csvfile:
    reader = csv.reader(csvfile)
    train_list = [row[1] for row in reader]
with open('./csv/validation.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    validation_list = [row['patient'] for row in reader]
with open('./csv/test.csv','r') as csvfile:
    reader = csv.DictReader(csvfile)
    test_list = [row['patient'] for row in reader]

dataset['outcome'] = pd.to_numeric(dataset['outcome'],errors='coerce')
array_OG = dataset.values
print(array_OG.shape)
train_list = train_list[1:]
validation_list = validation_list[0:]
test_list = test_list[0:]
#print(test_list)
#print(train_list)
#print(validation_list)

def cat_str(num_list):
    n_list = []
    for i in num_list:
        temp = i[12:]
        n_list.append(temp)
    n_list = [int(x) for x in n_list]
    return n_list

train_list = cat_str(train_list)
validation_list = cat_str(validation_list)
test_list = cat_str(test_list)

#print(train_list)
#print(validation_list)
#print(test_list)
#print(len(test_list))

train_feature = []
validate_feature = []
test_feature = []
count = 1
for i in range(len(array_OG)):
    num = i + 1
    if num in train_list:
        train_feature.append(array_OG[i])
    elif num in validation_list:
        validate_feature.append(array_OG[i])
    elif num in test_list:
        #print(count)
        count = count + 1
        test_feature.append(array_OG[i])
        #print(num)
        #print(array_OG[i,6096])
        
train_feature = np.array(train_feature)
validate_feature = np.array(validate_feature)
test_feature = np.array(test_feature)

train_feature = pd.DataFrame(train_feature)
train_feature.dropna(axis=1, thresh=2, inplace=True)
train_feature.dropna(how='all',thresh = 20,inplace=True)
train_feature = np.array(train_feature)
wh_inf = np.isinf(train_feature)
train_feature[wh_inf]=0
wh_nan = np.isnan(train_feature)
train_feature[wh_nan]=0

validate_feature = pd.DataFrame(validate_feature)
validate_feature.dropna(axis=1, thresh=2, inplace=True)
#validate_feature.dropna(how='all',thresh = 20,inplace=True)
validate_feature = np.array(validate_feature)
wh_inf = np.isinf(validate_feature)
validate_feature[wh_inf]=0
wh_nan = np.isnan(validate_feature)
validate_feature[wh_nan]=0

test_feature = pd.DataFrame(test_feature)
test_feature.dropna(axis=1, thresh=2, inplace=True)
#test_feature.dropna(how='all',thresh = 20,inplace=True)
test_feature = np.array(test_feature)
wh_inf = np.isinf(test_feature)
test_feature[wh_inf]=0
wh_nan = np.isnan(test_feature)
test_feature[wh_nan]=0

#only use image features
X_train = train_feature[:,:6093]
Y_train = train_feature[:,6093]
Y_train = Y_train.astype('int32')

X_validate = validate_feature[:,:6093]
Y_validate = validate_feature[:,6093]
Y_validate = Y_validate.astype('int32')

X_test = test_feature[:,:6093]
Y_test = test_feature[:,6093]
Y_test = Y_test.astype('int32')
seed = 7

np.random.seed(seed)
np.random.shuffle(X_train) 
np.random.seed(seed)
np.random.shuffle(Y_train)

print(Y_test)

(849, 6097)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1]


In [10]:
import csv
import pandas as pd
import numpy as np
from sklearn import model_selection
from tpot import TPOTClassifier
import tools

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
#from pyearth import Earth
from sklearn.cross_decomposition import PLSRegression


from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from skfeature.function.statistical_based import t_score
from skfeature.function.statistical_based import gini_index
from skfeature.function.similarity_based import fisher_score
from skfeature.function.similarity_based import reliefF

from sklearn.feature_selection import mutual_info_classif
from skfeature.function.information_theoretical_based import LCSI
from skfeature.function.information_theoretical_based import MIM
from skfeature.function.information_theoretical_based import MIFS
from skfeature.function.information_theoretical_based import MRMR
from skfeature.function.information_theoretical_based import CIFE
from skfeature.function.information_theoretical_based import JMI
from skfeature.function.information_theoretical_based import CMIM
from skfeature.function.information_theoretical_based import ICAP
from skfeature.function.information_theoretical_based import DISR

from scipy.stats import wilcoxon

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# CLASSIFICATION METHODS
models = []
models.append(('GLM', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DT', DecisionTreeClassifier()))
models.append(('BY', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('BAG', BaggingClassifier()))
models.append(('NNet', MLPClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('BST', AdaBoostClassifier()))

seed = 7

In [11]:
#crossvalidation on trainset and select the best model on validation set, test on test set
import xlwt

num_fea = 50
sel = []
sel.append(('CHSQ', SelectKBest(chi2, k=num_fea)))
sel.append(('ANOVA', SelectKBest(f_classif, k=num_fea)))
sel.append(('TSCR', SelectKBest(t_score.t_score, k=num_fea)))
sel.append(('FSCR', SelectKBest(fisher_score.fisher_score, k=num_fea)))
sel.append(('RELF', SelectKBest(reliefF.reliefF, k=num_fea)))

output = open("endohand50.txt","w")
file = 'fea50.xls'


book = xlwt.Workbook()
sheet = book.add_sheet('train_avg_auc')
sheet_train = book.add_sheet('train_auc')
sheet_validate = book.add_sheet('validate_auc')
sheet_test = book.add_sheet('test_auc')


from sklearn.externals import joblib
#from imblearn.over_sampling import SMOTE
# 定义SMOTE模型，random_state相当于随机数种子的作用
kfold = model_selection.KFold(n_splits=5, random_state=seed)
r = 0
c = 0
for name, model in models:
    for kind, selection in sel:
        pipe = make_pipeline(MinMaxScaler(), selection, model)
        cv_results = model_selection.cross_val_score(pipe, X_train, Y_train, scoring='roc_auc', cv=kfold)
        
        sheet.write(r,c,cv_results.mean())

        print("###########################################")
        msg = "%s %s %s: %f (%f)\n" % ("TRAIN_AUC", kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        pipe.fit(X_train,Y_train)
        joblib.dump(pipe,'./handpkl/Endo'+name+kind+'50.pkl')
        
        Y_pred = pipe.predict(X_train)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))

        Y_pred = pipe.predict(X_validate)
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = pipe.predict(X_test)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_test, Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

###########################################
TRAIN_AUC CHSQ GLM: 0.800823 (0.029883)

Accuracy: 0.7303030303030303
Average Precision Score: 0.6234748117101059
Kappa: 0.4565338070389696
Hamming Loss: 0.2696969696969697
AUC: 0.728987228987229
Sensitivity: 0.7162162162162162
Specificity: 0.7417582417582418
Accuracy: 0.7831325301204819
Average Precision Score: 0.6864676752121268
Kappa: 0.5667053364269141
Hamming Loss: 0.21686746987951808
AUC0.7856725146198831
Sensitivity0.8157894736842105
Specificity0.7555555555555555
Accuracy: 0.6385542168674698
Average Precision Score: 0.5495781754018362
Kappa: 0.2836593785960875
Hamming Loss: 0.3614457831325301
AUC: 0.6441520467836257
Sensitivity0.7105263157894737
Specificity0.5777777777777777


/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib

###########################################
TRAIN_AUC ANOVA GLM: 0.797512 (0.021004)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

Accuracy: 0.9818181818181818
Average Precision Score: 0.9746578965328966
Kappa: 0.9631531933899062
Hamming Loss: 0.01818181818181818
AUC: 0.9803608553608554
Sensitivity: 0.9662162162162162
Specificity: 0.9945054945054945
Accuracy: 0.6024096385542169
Average Precision Score: 0.5160063002434185
Kappa: 0.18748145950756456
Hamming Loss: 0.39759036144578314
AUC0.5923976608187135
Sensitivity0.47368421052631576
Specificity0.7111111111111111
Accuracy: 0.6867469879518072
Average Precision Score: 0.5897273303741281
Kappa: 0.36641221374045807
Hamming Loss: 0.3132530120481928
AUC: 0.6824561403508773
Sensitivity0.631578947368421
Specificity0.7333333333333333
###########################################
TRAIN_AUC TSCR RF: 0.756581 (0.024913)

Accuracy: 0.9803030303030303
Average Precision Score: 0.9742747532221216
Kappa: 0.9600447052249231
Hamming Loss: 0.019696969696969695
AUC: 0.9783561033561033
Sensitivity: 0.9594594594594594
Specificity: 0.9972527472527473
Accuracy: 0.6867469879518072
Average Pre

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)


###########################################
TRAIN_AUC ANOVA BST: 0.732051 (0.037656)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [51 54 57] are constant.
  UserWarning)


Accuracy: 0.8348484848484848
Average Precision Score: 0.745615344393846
Kappa: 0.6673079413233689
Hamming Loss: 0.16515151515151516
AUC: 0.8348121473121473
Sensitivity: 0.8344594594594594
Specificity: 0.8351648351648352
Accuracy: 0.7108433734939759
Average Precision Score: 0.6134978510533361
Kappa: 0.4151497357604228
Hamming Loss: 0.2891566265060241
AUC0.7067251461988304
Sensitivity0.6578947368421053
Specificity0.7555555555555555
Accuracy: 0.6626506024096386
Average Precision Score: 0.5675666655541834
Kappa: 0.3204678362573099
Hamming Loss: 0.3373493975903614
AUC: 0.660233918128655
Sensitivity0.631578947368421
Specificity0.6888888888888889
###########################################
TRAIN_AUC TSCR BST: 0.737287 (0.037749)

Accuracy: 0.8545454545454545
Average Precision Score: 0.7707633219729995
Kappa: 0.7072629828127888
Hamming Loss: 0.14545454545454545
AUC: 0.8551937926937927
Sensitivity: 0.8614864864864865
Specificity: 0.8489010989010989
Accuracy: 0.6867469879518072
Average Precision

In [12]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_validate = scaler.fit_transform(X_validate)
X_test = scaler.fit_transform(X_test)

# WILCOXON SCORE FUNCTION
def takeSecond(elem):
    return elem[1]
def WLCX(data, target, n_selected_features):
    pval = []
    for num in range(len(data[1])):
        x = data[:,num]
        pval.append([num, wilcoxon(x,target)[1]])
    pval.sort(key=takeSecond)
    idx = []
    for i in range(n_selected_features):
        idx.append(pval[i][0])
    return idx

# MULTIVARIATE FEATURE SELECTION X CLASSIFICATION (10 fold CV)

# print('BEFORE')
MV_sel = []
MV_sel.append(('WLCX', WLCX(X_train, Y_train, n_selected_features=num_fea)))
print('WLCX')
for name, model in models:
    for kind, idx in MV_sel:
        # X_sel = X[:, idx[0:num_fea]]
        X_test_ = X_test[:,idx[0:num_fea]]
        X_validate_ = X_validate[:,idx[0:num_fea]]
        X_train_ = X_train[:, idx[0:num_fea]]
        # X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_sel, Y, test_size=validation_size, random_state=seed)
        #kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, cv=kfold, scoring='roc_auc')
        
        sheet.write(r,c,cv_results.mean())
        
        msg = "%s %s: %f (%f)\n" % (kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_, Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'50.pkl')
        
        Y_pred = model.predict(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))
        
        Y_pred = model.predict(X_validate_)    
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_test,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

WLCX
WLCX GLM: 0.749474 (0.047473)

Accuracy: 0.7090909090909091
Average Precision Score: 0.6073914823914823
Kappa: 0.39514281350236746
Hamming Loss: 0.2909090909090909
AUC: 0.6920849420849421
Sensitivity: 0.527027027027027
Specificity: 0.8571428571428571
Accuracy: 0.7108433734939759
Average Precision Score: 0.6144951322317133
Kappa: 0.41273584905660377
Hamming Loss: 0.2891566265060241
AUC0.7046783625730995
Sensitivity0.631578947368421
Specificity0.7777777777777778
Accuracy: 0.7108433734939759
Average Precision Score: 0.6192929613189601
Kappa: 0.4053731343283582
Hamming Loss: 0.2891566265060241
AUC0.6985380116959066
Sensitivity0.5526315789473685
Specificity0.8444444444444444
WLCX LDA: 0.701108 (0.028103)

Accuracy: 0.7181818181818181
Average Precision Score: 0.6160777356089856
Kappa: 0.4170718735754444
Hamming Loss: 0.2818181818181818
AUC: 0.7037978912978913
Sensitivity: 0.5641891891891891
Specificity: 0.8434065934065934
Accuracy: 0.6867469879518072
Average Precision Score: 0.592807319

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

WLCX KNN: 0.709373 (0.051433)

Accuracy: 0.7772727272727272
Average Precision Score: 0.6800686630944934
Kappa: 0.5461859412127903
Hamming Loss: 0.22272727272727272
AUC: 0.770938520938521
Sensitivity: 0.7094594594594594
Specificity: 0.8324175824175825
Accuracy: 0.7228915662650602
Average Precision Score: 0.6319725362429756
Kappa: 0.4313375037235626
Hamming Loss: 0.27710843373493976
AUC0.7116959064327485
Sensitivity0.5789473684210527
Specificity0.8444444444444444
Accuracy: 0.7228915662650602
Average Precision Score: 0.625374899312756
Kappa: 0.44066803398769405
Hamming Loss: 0.27710843373493976
AUC0.7198830409356725
Sensitivity0.6842105263157895
Specificity0.7555555555555555
WLCX DT: 0.598607 (0.052393)

Accuracy: 0.9454545454545454
Average Precision Score: 0.932923832923833
Kappa: 0.8884716485167105
Hamming Loss: 0.05454545454545454
AUC: 0.9391891891891893
Sensitivity: 0.8783783783783784
Specificity: 1.0
Accuracy: 0.6265060240963856
Average Precision Score: 0.5358467362272055
Kappa: 0.23

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/harrisonbai/miniconda3/lib/

WLCX SVM: 0.757169 (0.042634)

Accuracy: 0.6909090909090909
Average Precision Score: 0.5910373608903021
Kappa: 0.34931374444229646
Hamming Loss: 0.3090909090909091
AUC: 0.6677123552123553
Sensitivity: 0.44256756756756754
Specificity: 0.8928571428571429
Accuracy: 0.6144578313253012
Average Precision Score: 0.5243143627140139
Kappa: 0.1867728107777098
Hamming Loss: 0.3855421686746988
AUC0.5891812865497076
Sensitivity0.2894736842105263
Specificity0.8888888888888888
Accuracy: 0.6506024096385542
Average Precision Score: 0.5628147263978515
Kappa: 0.26458906202260923
Hamming Loss: 0.3493975903614458
AUC0.626608187134503
Sensitivity0.34210526315789475
Specificity0.9111111111111111
WLCX BAG: 0.720190 (0.058295)

Accuracy: 0.9303030303030303
Average Precision Score: 0.912789819039819
Kappa: 0.8571267223853625
Hamming Loss: 0.0696969696969697
AUC: 0.9226128601128601
Sensitivity: 0.847972972972973
Specificity: 0.9972527472527473
Accuracy: 0.6867469879518072
Average Precision Score: 0.5907974001268

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

WLCX NNet: 0.751367 (0.045165)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.7181818181818181
Average Precision Score: 0.618810220648456
Kappa: 0.41328286304198203
Hamming Loss: 0.2818181818181818
AUC: 0.7006422631422631
Sensitivity: 0.5304054054054054
Specificity: 0.8708791208791209
Accuracy: 0.7349397590361446
Average Precision Score: 0.6447579792855633
Kappa: 0.4571938168846611
Hamming Loss: 0.26506024096385544
AUC0.7248538011695905
Sensitivity0.6052631578947368
Specificity0.8444444444444444
Accuracy: 0.7710843373493976
Average Precision Score: 0.6913607242035291
Kappa: 0.5302353291629431
Hamming Loss: 0.2289156626506024
AUC0.7602339181286549
Sensitivity0.631578947368421
Specificity0.8888888888888888
WLCX RF: 0.724513 (0.063143)

Accuracy: 0.9378787878787879
Average Precision Score: 0.919178739868395
Kappa: 0.8730221296244158
Hamming Loss: 0.06212121212121212
AUC: 0.9316899316899316
Sensitivity: 0.8716216216216216
Specificity: 0.9917582417582418
Accuracy: 0.6867469879518072
Average Precision Score: 0.5928073195035782
Kappa: 0.35582089552238805
Ha

In [13]:
MV_sel = []
MV_sel.append(('MIM', MIM.mim(X_train, Y_train, n_selected_features=num_fea)))
print('MIM')
MV_sel.append(('MIFS', MIFS.mifs(X_train, Y_train, n_selected_features=num_fea)))
print('MIFS')
MV_sel.append(('MRMR', MRMR.mrmr(X_train, Y_train, n_selected_features=num_fea)))
print('MRMR')
MV_sel.append(('CIFE', CIFE.cife(X_train, Y_train, n_selected_features=num_fea)))
print('CIFE')
MV_sel.append(('JMI', JMI.jmi(X_train, Y_train, n_selected_features=num_fea)))
print('JMI')
MV_sel.append(('CMIM', CMIM.cmim(X_train, Y_train, n_selected_features=num_fea)))
print('CMIM')
MV_sel.append(('ICAP', ICAP.icap(X_train, Y_train, n_selected_features=num_fea)))
print('ICAP')
MV_sel.append(('DISR', DISR.disr(X_train, Y_train, n_selected_features=num_fea)))

MIM
MIFS
MRMR
CIFE
JMI
CMIM
ICAP


In [14]:
for name, model in models:
    for kind, idx in MV_sel:
        #print(idx[0:num_fea][0])
        # X_sel = X[:, idx[0:num_fea]]
        X_test_ = X_test[:,idx[0:num_fea][0]]
        X_validate_ = X_validate[:,idx[0:num_fea][0]]
        X_train_ = X_train[:, idx[0:num_fea][0]]
        # X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_sel, Y, test_size=validation_size, random_state=seed)
        #kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X_train_, Y_train, cv=kfold, scoring='roc_auc')
        
        sheet.write(r,c,cv_results.mean())
        
        msg = "%s %s: %f (%f)\n" % (kind, name, cv_results.mean(), cv_results.std())
        print(msg)
        output.write(msg)
        model.fit(X_train_, Y_train)
        joblib.dump(model,'./handpkl/Endo'+name+kind+'50.pkl')
        
        Y_pred = model.predict(X_train_)
        print("Accuracy: " + repr(accuracy_score(Y_train, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_train, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_train, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_train, Y_pred)))
        print("AUC: " + repr(roc_auc_score(Y_train, Y_pred)))
        print("Sensitivity: " + repr(recall_score(Y_train, Y_pred)))
        tn, fp, fn, tp = confusion_matrix(Y_train, Y_pred).ravel()
        print("Specificity: " + repr(tn / (tn + fp)))
        
        sheet_train.write(r,c,roc_auc_score(Y_train, Y_pred))
        
        Y_pred = model.predict(X_validate_)    
        print("Accuracy: " + repr(accuracy_score(Y_validate, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_validate, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_validate, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_validate, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_validate,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_validate,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_validate,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_validate.write(r,c,roc_auc_score(Y_validate,Y_pred))
        
        Y_pred = model.predict(X_test_)
        print("Accuracy: " + repr(accuracy_score(Y_test, Y_pred)))
        print("Average Precision Score: " + repr(average_precision_score(Y_test, Y_pred)))
        print("Kappa: " + repr(cohen_kappa_score(Y_test, Y_pred)))
        print("Hamming Loss: " + repr(hamming_loss(Y_test, Y_pred)))
        print("AUC"+repr(roc_auc_score(Y_test,Y_pred)))
        print("Sensitivity" + repr(recall_score(Y_test,Y_pred)))
        tn,fp,fn,tp = confusion_matrix(Y_test,Y_pred).ravel()
        print("Specificity" + repr(tn/(tn+fp)))
        
        sheet_test.write(r,c,roc_auc_score(Y_test,Y_pred))
        
        r = r + 1
    c = c + 1
    r = 0

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

MIM GLM: 0.803367 (0.024754)

Accuracy: 0.740909090909091
Average Precision Score: 0.6357486064233469
Kappa: 0.4750990642382751
Hamming Loss: 0.2590909090909091
AUC: 0.7370247995247996
Sensitivity: 0.6993243243243243
Specificity: 0.7747252747252747
Accuracy: 0.7590361445783133
Average Precision Score: 0.6739378566899176
Kappa: 0.5065398335315101
Hamming Loss: 0.24096385542168675
AUC0.7491228070175439
Sensitivity0.631578947368421
Specificity0.8666666666666667
Accuracy: 0.7710843373493976
Average Precision Score: 0.6759101182056161
Kappa: 0.5398307557630581
Hamming Loss: 0.2289156626506024
AUC0.7704678362573099
Sensitivity0.7631578947368421
Specificity0.7777777777777778
MIFS GLM: 0.834626 (0.040104)

Accuracy: 0.7651515151515151
Average Precision Score: 0.6653852779314035
Kappa: 0.5214885774692686
Hamming Loss: 0.23484848484848486
AUC: 0.7586872586872587
Sensitivity: 0.6959459459459459
Specificity: 0.8214285714285714
Accuracy: 0.7228915662650602
Average Precision Score: 0.631972536242975

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

Accuracy: 0.7787878787878788
Average Precision Score: 0.6795596295596296
Kappa: 0.5514133551823023
Hamming Loss: 0.22121212121212122
AUC: 0.7748366498366498
Sensitivity: 0.7364864864864865
Specificity: 0.8131868131868132
Accuracy: 0.7710843373493976
Average Precision Score: 0.6836532733806038
Kappa: 0.5341211225997046
Hamming Loss: 0.2289156626506024
AUC0.7643274853801169
Sensitivity0.6842105263157895
Specificity0.8444444444444444
Accuracy: 0.6506024096385542
Average Precision Score: 0.5622413097365538
Kappa: 0.31986436846566824
Hamming Loss: 0.3493975903614458
AUC0.6654970760233918
Sensitivity0.8421052631578947
Specificity0.4888888888888889
JMI GLM: 0.798739 (0.030733)

Accuracy: 0.7363636363636363
Average Precision Score: 0.6301504914004914
Kappa: 0.467741935483871
Hamming Loss: 0.2636363636363636
AUC: 0.7341661716661716
Sensitivity: 0.7128378378378378
Specificity: 0.7554945054945055
Accuracy: 0.6626506024096386
Average Precision Score: 0.5906520820122596
Kappa: 0.28227300802964805
H

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

...

/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

DISR NNet: 0.799491 (0.046842)



/Users/harrisonbai/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy: 0.740909090909091
Average Precision Score: 0.6352147602147602
Kappa: 0.4764237599510105
Hamming Loss: 0.2590909090909091
AUC: 0.7382870507870507
Sensitivity: 0.7128378378378378
Specificity: 0.7637362637362637
Accuracy: 0.7831325301204819
Average Precision Score: 0.6884908053265694
Kappa: 0.564938846825859
Hamming Loss: 0.21686746987951808
AUC0.7836257309941521
Sensitivity0.7894736842105263
Specificity0.7777777777777778
Accuracy: 0.7108433734939759
Average Precision Score: 0.6114241655617686
Kappa: 0.42461005199306767
Hamming Loss: 0.2891566265060241
AUC0.7149122807017545
Sensitivity0.7631578947368421
Specificity0.6666666666666666
MIM RF: 0.762104 (0.030954)

Accuracy: 0.9818181818181818
Average Precision Score: 0.9761379386379386
Kappa: 0.9631298648095044
Hamming Loss: 0.01818181818181818
AUC: 0.9800452925452925
Sensitivity: 0.9628378378378378
Specificity: 0.9972527472527473
Accuracy: 0.6746987951807228
Average Precision Score: 0.5802093824213261
Kappa: 0.3617203076046711
Ham

Sensitivity0.6842105263157895
Specificity0.5333333333333333
JMI BST: 0.740990 (0.046187)

Accuracy: 0.8378787878787879
Average Precision Score: 0.7514806384371602
Kappa: 0.6725886912820133
Hamming Loss: 0.1621212121212121
AUC: 0.8366127116127117
Sensitivity: 0.8243243243243243
Specificity: 0.8489010989010989
Accuracy: 0.5542168674698795
Average Precision Score: 0.46905516804058334
Kappa: 0.03700219504546898
Hamming Loss: 0.4457831325301205
AUC0.5172514619883042
Sensitivity0.07894736842105263
Specificity0.9555555555555556
Accuracy: 0.6385542168674698
Average Precision Score: 0.5470513633481293
Kappa: 0.24408014571949
Hamming Loss: 0.3614457831325301
AUC0.6175438596491228
Sensitivity0.3684210526315789
Specificity0.8666666666666667
CMIM BST: 0.755933 (0.030871)

Accuracy: 0.8590909090909091
Average Precision Score: 0.7866293713067907
Kappa: 0.7136218576787414
Hamming Loss: 0.1409090909090909
AUC: 0.8548967923967923
Sensitivity: 0.8141891891891891
Specificity: 0.8956043956043956
Accuracy: 

In [15]:
book.save(file)
output.close()

In [ ]:
prob 
